In [ ]:
from langchain_community.llms import Ollama
from langchain.chains import ConversationChain
from langgraph.graph import StateGraph,END
from typing import TypedDict,Annotated,Sequence
from langchain_core.messages import BaseMessage,ToolMessage,SystemMessage,HumanMessage,AIMessage
from langgraph.graph.message import add_messages
from sever import Exploratory_analysis

In [10]:
class Agent_converse(TypedDict):
    message : Annotated[Sequence[BaseMessage], add_messages]
    pathfile : str

class Agent_summerize(TypedDict):
    content : str

llm = Ollama(model="llama3.2")

def choice(state: Agent_converse) -> Agent_converse:
    """Talk with agent"""
    messages = list(state["message"])  # copy existing conversation

    # Extract last user input
    last_user_msg = messages[-1].content if messages else ""

    # Case 1: detect file path setting
    if last_user_msg.lower().startswith("set file path"):
        path = last_user_msg.split("set file path", 1)[1].strip()
        state["pathfile"] = path
        messages.append(AIMessage(content=f"✅ File path stored: {path}"))
        state["message"] = messages
        return state

    # Case 2: analysis request
    if "analyse" and 'data' in last_user_msg.lower() and state["pathfile"]:
        # tool_result = Exploratory_analysis(state["pathfile"])
        # summary_prompt = f"Summarize this exploratory analysis:\n{tool_result}"
        # ai_response = llm.invoke(summary_prompt)
        # messages.append(AIMessage(content=ai_response))
        # state["message"] = messages
        # return state
        return 'explore'

    if 'exit' in last_user_msg.lower() and state["pathfile"]:
        return 'end'

In [38]:
def sumerize(state: Agent_converse)->Agent_converse:
    # systemprompt=SystemMessage(content=f'summerize this in a report {Exploratory_analysis(file_path=r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\protein.csv")}')
    systemprompt='hello'
    response = llm.invoke(systemprompt)
    print("Full Response:", )
    if isinstance(response, AIMessage):
        print("AI says:", response.content)
        return {"message": [response]}
    else:
        print("Unexpected response type:", type(response))
        return {"message": []}

In [39]:
graph=StateGraph(Agent_converse)
graph.add_node('start',sumerize)
graph.set_entry_point('start')
graph.add_edge('start',END)
app=graph.compile()

In [40]:
from IPython.display import Image,display
display(Image(app.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [41]:
res=app.invoke({'message':'','pathfile':''})
print('f',res)

Full Response:
Unexpected response type: <class 'str'>
f {'message': [HumanMessage(content='', additional_kwargs={}, response_metadata={}, id='7f930290-c21a-4321-9485-9f2aef7dafbe')], 'pathfile': ''}
